# Authors:
##### Maria Musiał : 156062
##### Joanna Szczublińska : 156xxx
##### Wiktoria Szarzyńska : 156xxx

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.data import Dataset 
import torch
import torchvision.models as models
import PIL.Image
import string

In [2]:
root_dir = "put_jetbot_dataset/dataset/"
samples=[]
folders = [name for name in os.listdir(root_dir)
           if os.path.isdir(os.path.join(root_dir, name))]


class JetBotDataset(Dataset):
    """
    Dataset that reads images and forward, left values from csv (from root_dir), uses transformations externally defined.
    Returns image(rgb), [forward, left] tensor
    """
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform
        
        for folder_name in folders:
            file_name = folder_name + ".csv"
            csv_path = os.path.join(root_dir, file_name)
            folder_path = os.path.join(root_dir, folder_name)
            # print(folder_name, file_name, csv_path, folder_path)

            df = pd.read_csv(csv_path, header=None, dtype={"0":"int32", "1":"float64", "2":"float64"})
            df.columns = ["filename", "forward", "left"]

            for _, row in df.iterrows():
                img_path = os.path.join(folder_path, str(row['filename'])[:-2].zfill(4)+".jpg")     # row[filename gives number of photo]; zfill gives leading zeros
                self.samples.append((img_path, float(row["forward"]), float(row["left"])))
            
    def __len__(self):
        return len(self.samples)
        
    def __getitem__(self, idx):
        img_path, forward, left = self.samples[idx]
        image = PIL.Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor([forward, left], dtype=torch.float32)
    
    
# Transformations: resize to 224x224, color jitter, normalize for rgb values predefined using imagenet 
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ColorJitter(0.3, 0.3, 0.3, 0.3),  #color transofmrations
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])    
    
    
dataset = JetBotDataset(root_dir, transform=transform)

### Divide to train and test

In [3]:
from torch.utils.data import random_split

generator = torch.Generator().manual_seed(44)
num_test = int(len(dataset) * 0.2)
train_dataset, test_dataset = random_split(dataset, [len(dataset) - num_test, num_test], generator= generator)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

### Resnet transfer learning

In [4]:
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512,2)
for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True
device = torch.device('cpu')
model = model.to(device)

c:\Users\mary9\anaconda3\envs\cv_lab\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mary9\anaconda3\envs\cv_lab\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Train model to 2 output: forward, left

In [5]:
import torch.nn.functional as functional
import torch.optim as optim


epochs = 70
best_model_path = 'best_model_jetbot.pth'
best_loss = 1e9
optimizer = optim.Adam(model.parameters())

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    
    for images, targets in iter(train_loader):
        images = images.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = functional.mse_loss(outputs, targets)
        train_loss += float(loss)
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for images, targets in iter(test_loader):
            images = images.to(device)
            targets = targets.to(device)
            outputs = model(images)
            loss = functional.mse_loss(outputs, targets)      
            test_loss += float(loss)
    test_loss /= len(test_loader)
        
    print(f"Train: {train_loss} -- Test: {test_loss}")
    if test_loss < best_loss:
        torch.save(model.state_dict(), best_model_path)
        best_loss = test_loss
        

Train: 0.261554937912777 -- Test: 0.19484505971011362
Train: 0.2039790159736776 -- Test: 0.18677932700436367
Train: 0.20310645433920724 -- Test: 0.19147536178168498
Train: 0.1955783085428836 -- Test: 0.1709758889518286
Train: 0.1922412280763876 -- Test: 0.1779511546814128
Train: 0.1972786026033763 -- Test: 0.17930755125064599
Train: 0.18912224242112774 -- Test: 0.1846361802988931
Train: 0.19318617091603743 -- Test: 0.16711553733207676
Train: 0.18947211219358587 -- Test: 0.16486569549888372
Train: 0.1915646972569081 -- Test: 0.20517855157193385
Train: 0.19719727788665314 -- Test: 0.17075242839361493
Train: 0.19324008948433463 -- Test: 0.19224801500769037
Train: 0.19356461781247644 -- Test: 0.1933968582827794
Train: 0.19359043006673554 -- Test: 0.16803123094141484
Train: 0.1866146765222309 -- Test: 0.17639752616615673
Train: 0.18836558166749 -- Test: 0.18239621227901232
Train: 0.19432253206985584 -- Test: 0.20218365286525927
Train: 0.19258865667936517 -- Test: 0.17594526291481757
Train: 

KeyboardInterrupt: 